<h1 p align='center'>Previsão de variáveis alvo contínuas com a Análise de Regressão</h1>
<p align='center'><img src=https://www.tibco.com/sites/tibco/files/media_entity/2020-09/regression-analysis-diagram.svg></p>

Modelos de regressão são usados para prever variáveis alvo em escala contínua, o que os torna atraentes para abordar muitas questões da ciência. Eles também têm aplicações na indústria, como entender as relações entre variáveis, avaliar tendências ou fazer previsões. Um exemplo é prever as vendas de uma empresa nos próximos meses.

\begin{align*}
y = x_{0} + w_{1}x
\end{align*}